In [8]:
import ee
import geemap.foliumap as geemap

In [9]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VNCuD1fdS64q1EwHjRqQDVB7U58pK56plFJyrvXsV2CWkmSYZtmP9Y

Successfully saved authorization token.


In [12]:
ee.Initialize()

In [13]:
zone = ee.FeatureCollection("users/nachogbn/pnatural_23030")

In [14]:
m = geemap.Map()
m.centerObject(zone)
m.addLayer(zone)
m

In [28]:
def get_NDVI(img):
    return img.addBands(img.normalizedDifference(["B4","B3"]).rename("NDVI"))

In [29]:
l7collection = (
    ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
    .filterBounds(zone)
    .filterDate('2008-01-01', '2018-01-01')
)

In [30]:
l7nvdi = l7collection.map(get_NDVI)

In [31]:
composite = l7nvdi.qualityMosaic('NDVI').clip(zone)

band = composite.select('NDVI')

In [32]:
m2 = geemap.Map()

m2.centerObject(zone, zoom=10)

vis_params = {'min': 0, 'max': 1, 'palette': ['beb261', '8ffff9', '1cd810']}

m2.addLayer(band, vis_params, 'NDVI')

m2

In [33]:
bands = []

years = range(2008, 2018)

for year in years:
    l7collection = (
    ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
    .filterBounds(zone)
    .filterDate(str(year) + '-01-01', str(year) + '-12-31')
    )
    
    l7nvdi = l7collection.map(get_NDVI)
    
    composite = l7nvdi.qualityMosaic('NDVI').clip(zone)

    band = composite.select('NDVI')
    
    bands.append(band)

In [34]:
m3 = geemap.Map()

m3.centerObject(zone, zoom=10)

vis_params = {'min': 0, 'max': 1, 'palette': ['beb261', '8ffff9', '1cd810']}

for position, band in enumerate (bands):
    m2.addLayer(band, vis_params, 'NDVI_' + str(2000 + position))

m3

In [35]:
lat = 37.0311
lon = -3.3259
point = ee.Geometry.Point([lon, lat])

In [36]:
#Con esto estamos juntando todas la bandas en una sola imagen porque es como lo necesita el programa para poder ver la tendencia que tiene el NDVI a lo largo de los años. Lo que hace es mulitplicas los valores para tener un valor entre -1 y 1 para ver si sube o baja
stack_collection = ee.ImageCollection(bands)
ndvis = stack_collection.toBands()
    

In [37]:
ndvi_trend = ndvis.reduce(ee.Reducer.kendallsCorrelation())
tau = ndvi_trend.select('tau')

In [38]:
m2 = geemap.Map()

m2.centerObject(zone, zoom=10)

vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'orange', 'yellow', 'green']}

m2.addLayer(tau, vis_params, 'Mann-Kendal')

m2